In [1]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import torch
from torch.autograd import Variable

import s3fs
from pyarrow.parquet import ParquetDataset
import io
import re
import string
import tqdm
import numpy as np
import pandas as pd

import random
import os, errno
import sys

In [4]:
#OUT_path_pairs = 's3://tk-dev-datalake2/risk/jerry.zhu/UBS_Embedding/Op_Log_masked_20221121_20221129.parquet'
data = pd.read_parquet('Op_Log_masked_v2.parquet')

In [5]:
user_counts = data['user_id'].value_counts().reset_index()
bottom_users = user_counts[user_counts['user_id']<=1]
bottom_users = bottom_users['index'].tolist()
data = data[~data['user_id'].isin(bottom_users)] 
print("Dataframe shape after filtering out singular operations is " + str(data.shape))

data['request_time'] = pd.to_datetime(data['request_time'], format='%Y-%m-%d %H:%M:%S')
data = data.sort_values(by=['request_time'])

print("There are " + str(data['operation'].nunique()) + " types of operations!")

Dataframe shape after filtering out singular operations is (240157, 7)
There are 249 types of operations!


In [ ]:
from sklearn.preprocessing import LabelEncoder

#x = data['operation'].to_numpy()
#x = OneHotEncoder().fit_transform(x.reshape(-1, 1)).toarray() # removed one-hot encoding since W2V takes categorical
y = data['user_id'].to_numpy()
z = data['request_time'].to_numpy()
 
# Creating a instance of label Encoder.
le = LabelEncoder()
x = le.fit_transform(data['operation'])
x += 1

In [ ]:
users = data['user_id'].unique()

In [ ]:
sequences = [] 
timestamps = []
index = 0

for user in users:
    uobs = []
    uobsTime = []
    previousIndex = -1
    for i in range(len(y)): 
        if user == y[i]:
            if previousIndex == -1:
                previousIndex = i
            timeElapsed = (z[i]-z[previousIndex]).astype('timedelta64[ms]').astype(int) 
            uobs.append(x[i])
            uobsTime.append(timeElapsed)
            previousIndex = i
    sequences.append(uobs)
    timestamps.append(uobsTime)
    index += 1
    print(index)
print("done")

In [3]:
with open('weights.npy', 'rb') as f:
    weights = np.load(f)

In [4]:
import pickle

with open('sequences', 'rb') as fp:  
    sequences = pickle.load(fp)
    
with open('timestamps', 'rb') as fp:  
    timestamps = pickle.load(fp)

In [30]:
def w2vembedding(sequences, weights):
    for seqIndex, sequence in enumerate(sequences):
        for opIndex, operation in enumerate(sequence):
            sequences[seqIndex][opIndex] = weights[operation - 1]
    return sequences

num_sequences = sequences
sequences = w2vembedding(sequences, weights)

IndexError: arrays used as indices must be of integer (or boolean) type

In [31]:
def window(sequences, window_size = 20): 
    windowed_sequences = []
    for sequence in sequences:
        if len(sequence) >= window_size:
            for i in range(len(sequence) - window_size):
                windowed_sequences.append(sequence[i:window_size+i])
    return windowed_sequences

In [32]:
windowed_sequences = window(sequences)
torched_windowed_sequences = torch.from_numpy(np.array(windowed_sequences))
torched_windowed_sequences = torched_windowed_sequences.type(torch.FloatTensor)

windowed_timestamps = window(timestamps)
torched_windowed_timestamps = torch.from_numpy(np.array(windowed_timestamps))
torched_windowed_timestamps  = torched_windowed_timestamps.type(torch.FloatTensor)

In [8]:
!pip uninstall numpy --yes
!pip install numpy

import numpy as np

Found existing installation: numpy 1.24.2
Uninstalling numpy-1.24.2:
  Successfully uninstalled numpy-1.24.2
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [10]:
from sklearn.model_selection import train_test_split

train_op, test_op, _, _ = train_test_split(np.array(windowed_sequences), np.array(windowed_sequences), test_size=0.15, random_state=42)
train_op = torch.from_numpy(train_op)
train_op = train_op.type(torch.FloatTensor)
test_op = torch.from_numpy(test_op)
test_op = test_op.type(torch.FloatTensor)

train_time, test_time, _, _ = train_test_split(np.array(windowed_timestamps), np.array(windowed_timestamps), test_size=0.15, random_state=42)
train_time = torch.from_numpy(train_time)
train_time = train_time.type(torch.FloatTensor)
test_time = torch.from_numpy(test_time)
test_time = test_time.type(torch.FloatTensor)


In [35]:
train_op = train_op.permute(1,0,2)
test_op = test_op.permute(1,0,2)
torched_windowed_sequences = torched_windowed_sequences.permute(1,0,2)
train_time = train_time.permute(1,0)
test_time = test_time.permute(1,0)
torched_windowed_timestamps= torched_windowed_timestamps.permute(1,0)

In [36]:
print(train_op.shape)
print(test_time.shape)
print(torched_windowed_sequences.shape)

torch.Size([135030, 20, 128])
torch.Size([23829, 20])
torch.Size([158859, 20, 128])


In [37]:
with open('train_op','wb') as f:
  torch.save(train_op, f)

with open('test_op','wb') as f:
  torch.save(test_op, f)

with open('train_time','wb') as f:
  torch.save(train_op, f)

with open('test_time','wb') as f:
  torch.save(test_op, f)

with open('torched_windowed_sequences','wb') as f:
  torch.save(torched_windowed_sequences, f)

with open('torched_windowed_timestamps','wb') as f:
  torch.save(torched_windowed_timestamps, f)